<div align="right" style="text-align:right"><i>Peter Norvig<br>2014, 2020</i></div>

# Cryptarithmetic Problems

On 28 April 2014 Gary Antonik had a [Numberplay column](https://web.archive.org/web/20191001030614/https://wordplay.blogs.nytimes.com/2014/04/28/num/) that quotes my friend [Bill Gosper](https://en.wikipedia.org/wiki/Bill_Gosper). (Gosper often presents more advanced puzzles in the [math-fun](http://mailman.xmission.com/cgi-bin/mailman/listinfo/math-fun) mailing list.)  This puzzle was:

> For the expression  `N U M + B E R = P L A Y`,
> - Which distinct numerals (each different) can be substituted for letters to make a valid expression?
> - How many solutions are there?

I [tackled](https://www.udacity.com/wiki/cs212/unit-2#rethinking-eval) this type of problem (also known as a [cryptarithmetic](http://mathworld.wolfram.com/Cryptarithmetic.html) or [alphametic](http://mathworld.wolfram.com/Alphametic.html) or [verbal arithmetic](https://en.wikipedia.org/wiki/Verbal_arithmetic) problem) in my Udacity class [CS 212](https://www.udacity.com/wiki/cs212/unit-2#cryptarithmetic). 



![](https://www.azquotes.com/picture-quotes/quote-when-in-doubt-use-brute-force-ken-thompson-131-46-05.jpg)

My initial approach follows Ken Thompson's adage,  "when in doubt, use brute force."

- There are only 10 factorial or 3.6 million permutations of 10 digits, so we can try them all.
- For each permutation, substitute the digits for the corresponding letters in the formula.
- Use Python's `eval` function to check if the resulting formula is a valid, true expression. 
- Report the one(s) that are.

The basic idea is simple, but there are a few complications to worry about:

- Python uses `==` for equality and `**` for exponentiation, but math notation  uses `=` and `^`.   I'll accept any of these, and translate from math to Python with `to_python`.
- A number with a leading zero digit (like `012`) is illegal (except that `0` by itself is ok).
- We'll have to catch divide-by-zero errors, as in `1/0` or `4/(3-2-1)`.
- In general, it is **unsafe** to eval a string that comes from a user, because a malicious user could inject malware. But eval-ing strings that we make ourselves is no worse than running code that we make ourselves. 
- Only uppercase letters are replaced by digits. So in `2 * B or not 2 * BE`, the `or` and `not` are Python keywords.
- Should I define a `solve` function to find one solution (faster) or all solutions (comprehensive)? I'll handle both use cases by defining `solve` to  yield solutions one at a time. You can quickly get the first one with `first` or all of them with `set`. 

# Defining `solve`

First some imports and type definitions:

In [1]:
from typing import Iterable, Callable, Tuple
from itertools import permutations
import re

Formula  = str # A formula in Math notation, like "NUM ^ BER = PLAY"
Pformula = str # A formula in Python notation, like "NUM ** BER == PLAY"
Solution = str # A formula with letters relaced by digits, like "356 + 742 = 1098"

Below we see that `solve` works by substituting each permutation of digits for the letters in the formula, and then reporting on the ones that are valid (ones that evaluate to true and have no number with a leading zero). Note that `solve` checks if a `to_python` version of the  formula is `valid`, and if it is, returns a substituted version of the original formula.

In [2]:
def solve(formula) -> Iterable[Solution]:
    """Given a formula like 'NUM + BER == PLAY', fill in digits to solve it."""
    letters  = all_letters(formula)
    pformula = to_python(formula)
    for digits in permutations('1234567890', len(letters)):
        if valid(subst(digits, letters, pformula)):
            yield subst(digits, letters, formula)
        
def subst(digits, letters, formula) -> Formula:
    """Substitute digits for letters in formula."""
    return formula.translate(str.maketrans(letters, cat(digits)))
        
def valid(pformula) -> bool:
    """A pformula is valid iff it has no leading zero, and evaluates to True."""
    try:
        return (not leading_zero(pformula)) and (eval(pformula) is True)
    except ArithmeticError:
        return False
    
def to_python(formula: Formula) -> Pformula:
    """Convert ' = ' and '^' to ' == ' and '**'."""
    return formula.replace(' = ', ' == ').replace('^', '**') 

def all_letters(formula: str) -> str: 
    """The set of letters in formula, in the form of an alphabetized string."""
    return cat(sorted(set(re.findall('[A-Z]', formula))))

def first(iterable): "First element"; return next(iter(iterable), None)
    
cat = ''.join # Function to concatenate strings
    
leading_zero = re.compile(r'\b0[0-9]').search # Function to check for illegal number

It would be good to have some unit tests for these functions, but I moved them to the end of this notebook, so that we can go right ahead and solve the problem:

In [3]:
first(solve('NUM + BER = PLAY'))

'587 + 439 = 1026'

That's one solution; how many are there all together? And how long does it take to find them?

In [4]:
%time solutions = set(solve('NUM + BER = PLAY'))
len(solutions)

CPU times: user 16.9 s, sys: 19.2 ms, total: 17 s
Wall time: 17 s


96

It takes 15 or 20 seconds to find 96 solutions, Here are ten of them:

In [5]:
list(solutions)[:10]

['452 + 637 = 1089',
 '589 + 437 = 1026',
 '879 + 426 = 1305',
 '749 + 286 = 1035',
 '756 + 342 = 1098',
 '673 + 425 = 1098',
 '439 + 587 = 1026',
 '432 + 657 = 1089',
 '423 + 675 = 1098',
 '429 + 876 = 1305']

# Defining `faster_solve`

Can we make `solve` faster? 

To answer the question, I start by profiling to see where the time is spent, using the ipython magic function `%prun`:

In [6]:
%prun first(solve('NUM + BER = PLAY'))

         3028913 function calls in 3.011 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   309270    1.779    0.000    1.833    0.000 {built-in method builtins.eval}
   453271    0.229    0.000    0.229    0.000 {method 'translate' of 'str' objects}
   453270    0.220    0.000    0.220    0.000 {method 'search' of 're.Pattern' objects}
        2    0.215    0.107    3.065    1.532 89248256.py:1(solve)
   453271    0.209    0.000    0.669    0.000 89248256.py:9(subst)
   453271    0.146    0.000    0.146    0.000 {built-in method maketrans}
   453270    0.128    0.000    2.182    0.000 89248256.py:13(valid)
   453272    0.084    0.000    0.084    0.000 {method 'join' of 'str' objects}
        1    0.000    0.000    3.065    3.065 89248256.py:28(first)
        1    0.000    0.000    3.065    3.065 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 89248256.py:24(all_letters)
        1    0.000    0

`%prun` tells us that over half the time is spent in `eval`, which makes sense because we call `eval` once for each pemutation of the digits. But we don't need to do that. We could call `eval` just once to create a Python function which we than **call** (not **eval**) for each permutation. That will be faster, because we won't have to re-eval the formula each time (parsing it and generating bytecode each time); it will be translated once into Python bytecode.

For `"NUM + BER = PLAY"`, the Python function could be:

    (lambda N,U,M,B,E,R,P,L,A,Y:
     N and B and P and (100*N + 10*U + M) + (100*B + 10*E + R) == (1000*P + 100*L + 10*A + Y))
     
Note that:
- The parameters of the function are the letters that appear in the formula.
- The word `NUM` in the math formula translates to `(100*N + 10*U + M)` in the Python function.
- The letters that start each word, `N, B` and `P`, cannot be zero, so we test them first. 
- A function might still divide by zero; that will have to be caught elsewhere.

Here is the code to do the translation:

In [7]:
def translate_formula(formula, verbose=False) -> Tuple[str, str]:
    """Translate formula into two values: (lambda function string, parameter letters)."""
    letters  = all_letters(formula)
    assert len(letters) <= 10, f'{len(letters)} letters is too many; only 10 allowed'
    firsts  = re.findall(r'\b([A-Z])[A-Z]', formula)
    body    = re.sub('[A-Z]+', translate_word, to_python(formula))
    return f'lambda {",".join(letters)}: {" and ".join([*firsts, body])}', letters

def translate_word(matchobj: re.Match) -> str:
    "Translate the matched word 'PLAY' to '(((P*10+L)*10+A)*10+Y))', e.g."
    word = matchobj.group()
    exponents = reversed(range(len(word)))
    terms = (f'{10 ** x}*{L}' if x > 0 else L 
             for x, L in zip(exponents, word))
    return '(' + ' + '.join(terms) + ')'

Some usage examples:

In [8]:
translate_formula("A + B = CD")

('lambda A,B,C,D: C and (A) + (B) == (10*C + D)', 'ABCD')

In [9]:
translate_formula("NUM + BER = PLAY")

('lambda A,B,E,L,M,N,P,R,U,Y: N and B and P and (100*N + 10*U + M) + (100*B + 10*E + R) == (1000*P + 100*L + 10*A + Y)',
 'ABELMNPRUY')

Now we're ready for the faster version of `solve`:

In [10]:
def faster_solve(formula: Formula) -> Iterable[Solution]:
    """Given a formula like 'NUM + BER == PLAY', fill in digits to solve it.
    This version precompiles the formula."""
    python_lambda, letters = translate_formula(to_python(formula))
    formula_fn = eval(python_lambda)
    for digits in permutations((1,2,3,4,5,6,7,8,9,0), len(letters)):
        try:
            if formula_fn(*digits) is True:
                yield subst(map(str, digits), letters, formula)
        except ArithmeticError: 
            pass           

In [11]:
first(faster_solve('NUM + BER = PLAY'))

'587 + 439 = 1026'

How fast is it? 

In [12]:
%time solutions2 = set(faster_solve('NUM + BER = PLAY'))

CPU times: user 1.11 s, sys: 3.16 ms, total: 1.11 s
Wall time: 1.11 s


About 15 times faster! Does it give the same solutions as `solve`?

In [13]:
assert solutions == solutions2

Yes, it does.

# More Example Formulas

Here are some classic examples as well as some I made up myself:

In [14]:
formulas = """
NUM + BER = PLAY
YOU = ME ^ 2
SEND + MORE = MONEY
FOUR + SCORE + 7 = YEARS + AGO
WRONG + WRONG = RIGHT
WRIGHT + WRIGHT = TO * FLY + FLIGHT
HALF + HALF = WHOLE
HALF + FIFTH + TENTH + TENTH + TENTH = WHOLE
BASE + BALL = GAMES
YOUR + YOU = HEART
EAT + THAT = APPLE
ALPHABET + LETTERS = SCRABBLE
POTATO + TOMATO = PUMPKIN
ODD * ODD = FREAKY
DOUBLE + DOUBLE + TOIL = TROUBLE
WASH + YOUR = HANDS
WEAR + A + MASK + WASH = SAFER
PERSON + WOMAN + MAN = CAMERA
TWO + TWENTY = TWELVE + TEN
AA + BB + CC = ABC
PI * R^2 = AREA
A^2 + B^2 = C^2
A^2 + BE^2 = BY^2 
AYH^2 + BEE^2 = SEE^2
RAMN = R^3 + RM^3 = N^3 + RX^3
MON-EY = EVIL^(1/2)
SIN^2 + COS^2 = UNITY
SPEED + LIMIT = FIFTY
TERRIBLE + NUMBER = THIRTEEN
SEVEN + SEVEN + SIX = TWENTY
EIGHT + EIGHT + TWO + ONE + ONE = TWENTY
ELEVEN + NINE + FIVE + FIVE = THIRTY
NINE + SEVEN + SEVEN + SEVEN = THIRTY
FOURTEEN + TEN + TEN + SEVEN = FORTYONE
HAWAII + IDAHO + IOWA + OHIO = STATES
VIOLIN * 2 + VIOLA = TRIO + SONATA
SEND + A + TAD + MORE = MONEY
ZEROES + ONES = BINARY
DCLIZ + DLXVI = MCCXXV
COUPLE + COUPLE = QUARTET
FISH + N + CHIPS = SUPPER
SATURN + URANUS + NEPTUNE + PLUTO = PLANETS
PLUTO not in {PLANETS}
EARTH + AIR + FIRE + WATER = NATURE
TWO * TWO = SQUARE
HIP * HIP = HURRAY
NORTH / SOUTH = EAST / WEST
NAUGHT ^ 2 = ZERO ^ 3
I + THINK + IT + BE + THINE = INDEED
DO + YOU + FEEL = LUCKY
WELL - DO + YOU = PUNK
NOW + WE + KNOW + THE = TRUTH
SORRY + TO + BE + A + PARTY = POOPER
SORRY + TO + BUST + YOUR = BUBBLE
STEEL + BELTED = RADIALS
ABRA + CADABRA + ABRA + CADABRA = HOUDINI
I + GUESS + THE + TRUTH = HURTS
LETS + CUT + TO + THE = CHASE
THATS + THE + THEORY = ANYWAY
TWO + TWO = FOUR
X / X = X
X + X = X
A^N + B^N = C^N and N > 1
ATOM^0.5 = A + TO + M
ALL + GLITTERS is not GOLD
ONE < TWO and FOUR < FIVE
ONE < TWO < THREE < TWO * TWO
(2 * BE or not 2 * BE) = THE + QUES-TION
sum(range(AA)) = BB
sum(range(POP)) = BOBO
ODD + ODD = EVEN
ROMANS+ALSO+MORE+OR+LESS+ADDED = LETTERS
FOUR+ONE = FIVE and ONE+ONE+ONE+ONE+ONE = FIVE
TEN+SEVEN+SEVEN+SEVEN+FOUR+FOUR+ONE = FORTY
NINETEEN+THIRTEEN+THREE+2*TWO+3*ONE = FORTYTWO
IN + ARCTIC + TERRAIN + AN + ANCIENT + EERIE + ICE + TRACT + I + ENTER + A + TRANCE = FLATIANA
ONE < TWO < THREE < SEVEN - THREE < THREE + TWO < THREE + THREE < SEVEN < SEVEN + ONE < THREE * THREE
ABCDEFGHIJ/JIHGFEDCBA = AI/IG
AN + ACCELERATING + INFERENTIAL + ENGINEERING + TALE + ELITE + GRANT + FEE + ET + CETERA = ARTIFICIAL + INTELLIGENCE
""".strip().splitlines()

In [15]:
def show(formulas: Iterable[Formula], sep='_'*90):
    """Solve all the formulas and show results."""
    for f in formulas:
        print(f'{sep}\n{f}\n{first(faster_solve(f))}')
    return len(formulas)
        
%time show(formulas)

__________________________________________________________________________________________
NUM + BER = PLAY
587 + 439 = 1026
__________________________________________________________________________________________
YOU = ME ^ 2
576 = 24 ^ 2
__________________________________________________________________________________________
SEND + MORE = MONEY
9567 + 1085 = 10652
__________________________________________________________________________________________
FOUR + SCORE + 7 = YEARS + AGO
9071 + 26014 + 7 = 34512 + 580
__________________________________________________________________________________________
WRONG + WRONG = RIGHT
37081 + 37081 = 74162
__________________________________________________________________________________________
WRIGHT + WRIGHT = TO * FLY + FLIGHT
413058 + 413058 = 82 * 769 + 763058
__________________________________________________________________________________________
HALF + HALF = WHOLE
9604 + 9604 = 19208
_____________________________________________

79

# Tests

Here are some unit tests for the functions defined above:

In [16]:
for solver in (solve, faster_solve):
    assert "1 + 2 = 3" in set(solver("A + B = C"))
    assert not set(solver("A + B = CDE"))
    assert set(solver("A * B = CA")) == {
        '5 * 3 = 15', '5 * 7 = 35', '4 * 6 = 24', '8 * 6 = 48', '2 * 6 = 12', '5 * 9 = 45'}

assert '359 + 847 = 1206' in set(faster_solve('NUM + BER = PLAY')) 

assert subst("123", "ABC", "A + B = C") == "1 + 2 = 3"

assert     valid("1 + 2 == 3")
assert not valid("1 + 2 == 4")
assert not valid("1/0 == 1/0")

assert to_python("A^B = C") == "A**B == C"

assert all_letters("A + B = C") == "ABC"

assert first("ABC") == "A"

assert cat(['A', 'B', 'C']) == "ABC"

assert     leading_zero('012')
assert not leading_zero('123')
assert not leading_zero('0')

assert translate_word(re.match('NUM', 'NUM')) == '(100*N + 10*U + M)'
assert translate_word(re.match('X', 'X')) == '(X)'

assert translate_formula("A + B = CD") == ('lambda A,B,C,D: C and (A) + (B) == (10*C + D)', 'ABCD')
assert translate_formula("NUM + BER = PLAY") == (
    'lambda A,B,E,L,M,N,P,R,U,Y: N and B and P and (100*N + 10*U + M) + (100*B + 10*E + R) == (1000*P + 100*L + 10*A + Y)',
    'ABELMNPRUY')